In [96]:
from __future__ import absolute_import, division, print_function
import tensorflow as tf
#from tensorflow import keras
from keras import backend as K
from keras import regularizers, Sequential
from keras.layers import Embedding, GlobalAveragePooling1D, Dense
from keras.callbacks import Callback, EarlyStopping
from keras.optimizers import SGD
from keras.models import load_model, model_from_json
from keras.utils import CustomObjectScope
from keras.initializers import glorot_uniform
import numpy as np
from numpy import ndarray
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import roc_auc_score
from sklearn.utils import class_weight
from sklearn.utils.validation import check_is_fitted, column_or_1d
from cophi_toolbox import preprocessing
import pandas as pd
from hatesonar import Sonar
sonar = Sonar()
from collections import defaultdict
from operator import itemgetter
import bisect
import matplotlib.pyplot as plt
import pickle
import importlib
import util
importlib.reload(util)
from util import make_w2v_embeddings, split_and_zero_padding, ManDist

In [89]:
json_file = open('sentiment_model.json', 'r')
loaded_model_json = json_file.read()
json_file.close()
with CustomObjectScope({'GlorotUniform': glorot_uniform()}):
    loaded_model_sentiment = model_from_json(loaded_model_json)
    loaded_model_sentiment.load_weights('sentiment_model.h5')
with open('encoder.pickle', 'rb') as handle:
    unserialized_data = pickle.load(handle)
encoder=unserialized_data['encoder']

# Proposed Model 

In [55]:
 #senti should be 0 or 1, 0 is negative, 1 is positive

def compareSim(input_sentence,model,datapath,senti):
  
    #retrive reviews based on input sentiment
    if senti==0:
        datafile=datapath+'/yelp_0.txt'
    elif senti==1:
        datafile=datapath+'/yelp_1.txt'

    with open(datafile) as f:
        datayelp=f.readlines()

    #data cleaning 
    for i in range(len(datayelp)):
        datayelp[i] = datayelp[i].replace('.','').replace('\n','').replace('!','')

    # find the top 3 similar reviews
    result_index=[]
    test_sentence_pairs=[]
    for i in range(len(datayelp)):
        test_sentence=(input_sentence,datayelp[i])
        test_sentence_pairs.append(test_sentence)


    embedding_dict = {}

    test_df = pd.DataFrame(test_sentence_pairs, columns = ['question1','question2'])
    for q in ['question1', 'question2']:
        test_df[q + '_n'] = test_df[q]

    test_df, embeddings = make_w2v_embeddings(embedding_dict, test_df, embedding_dim=300)

    X_test = split_and_zero_padding(test_df, 10)

    assert X_test['left'].shape == X_test['right'].shape

    preds = list(model.predict([X_test['left'], X_test['right']]))

    results = [(x, y, z) for (x, y), z in zip(test_sentence_pairs, preds)]
    results.sort(key=itemgetter(2), reverse=True)


    return results[0:3]

In [109]:
def check_senti(input_sentence):
    words = list(preprocessing.tokenize(input_sentence))
    words = list(map(lambda s: 'unknown' if s not in encoder.classes_ else s, words))
    encoder_classes = encoder.classes_.tolist()
    bisect.insort_left(encoder_classes, 'unknown')
    encoder.classes_ = np.array(encoder_classes)
    word_idx = np.array([encoder.transform(words)])
    
    if loaded_model_sentiment.predict_proba(word_idx)>0.6:
            senti = 1
    else:
            senti = 0

    return senti

def pred(input_sentence,model,datapath):
    
    class_value = sonar.ping(text=input_sentence)['top_class']
    if class_value == 'offensive_language':
        senti=check_senti(input_sentence)
        results=compareSim(input_sentence,model,datapath,senti)
    else:
        results='NO TOXIC'
    return results

In [120]:
modelpath='./en_SiameseLSTM.h5'
datapath='.'
model = load_model(modelpath, custom_objects={"ManDist": ManDist})
input_sen='The food is damn good'
print(pred(input_sen,model,datapath))

[('The food is damn good', 'this man was absolutely stellar ', array([0.9855283], dtype=float32)), ('The food is damn good', 'so yes i went there , the meal was good ', array([0.9409161], dtype=float32)), ('The food is damn good', 'the food was not super but good prices reasonable ', array([0.9269292], dtype=float32))]


In [57]:
toxic_sentences = pd.read_csv('toxic.csv')['Toxic'].tolist()
import random
toxic_validation = random.sample(toxic_sentences, 20)

In [58]:
modelpath='./en_SiameseLSTM.h5'
model = load_model(modelpath, custom_objects={"ManDist": ManDist})

In [110]:
datapath='.'
detoxic_list = []
correct_list = []
sentiment_list = []
for sentence in toxic_validation:
    senti = check_senti(sentence)
    ds = pred(sentence,model,datapath)
    detoxic_sentence = [ds[i][1] for i in range(3)]
    detoxic_list.append(detoxic_sentence)
    confid_score = [sonar.ping(text=ds[i][1])['classes'][1]['confidence'] for i in range(3)]
    sentiment_score = [check_senti(ds[i][1]) for i in range(3)]
    if  all(confid_score[i] < 0.8 for i in range(3)):
        correct_list.append(True)
    else:
        correct_list.append(False) 
    if any(sentiment_score[i] == senti for i in range(3)):
        sentiment_list.append(True)
    else:
        sentiment_list.append(False)

In [111]:
detoxic_df = pd.DataFrame(detoxic_list,columns = ['1st','2nd','3rd'])
origin_df = pd.DataFrame(toxic_validation, columns=['Origin'])
correct_df = pd.DataFrame(correct_list,columns=['Detoxicity'])
sentiment_df = pd.DataFrame(sentiment_list,columns=['Same Sentiment'])
output_df = pd.concat([origin_df,detoxic_df,sentiment_df,correct_df],axis=1)

In [112]:
output_df.to_csv('output.csv',index=False)
output_df

,Origin,1st,2nd,3rd,Same Sentiment,Detoxicity
0,"Last trip here, they have promos on their drin...",this location though has horribly slow service,huge fail for this type of venue,at this point they were open and would be for ...,True,True
1,Great show - laughed my ass off,"since i moved here , i 've been shopping at th...",i knew i should have taken my camera,so happy i found this place,True,True
2,"Boy, do these boys kick my ass every time I ...",the staff is always nice and they keep the sto...,it is a great hometown neighborhood bar with g...,great homemade hot breakfast sausage and a won...,True,True
3,"diarrhea lots, shit my pants once",extremely slow service and a rude waitress on ...,one table near us had to flag a waitress just ...,come on -- this is an `` asian bistro '',True,True
4,Fucking eat something,"servers are chatting , not paying attention to...",and there 's hardly any tables to sit at,both drinks were overly sweet and barely any a...,True,True
5,The service sucked ass,awful awful,absolutely the worst care in all my experience...,awful acoustics,True,True
6,there is a large dance floor for some shit k...,the apricot cream cheese ones are delicious,"this location is now my closest store , so i d...",so amazing with the sauce on the side,True,True
7,From the valet who is running his ass off to ...,the service is always top notch and customer s...,it was truly outstanding,always a treat to have dinner at papa j 's,True,True
8,The chips 'n salsa kicks ass,i really really want this place to do better,"okay , i get it , they work on commission , bu...",it was really embarrassing and every time he c...,True,True
9,"No sweat, I could give a shit less as long a...",it had no signs of being coated and sauted to ...,this store has such bad service,this place has none of them,True,True


# Benchmark Model

In [113]:
json_file_toxic = open('toxic_model.json', 'r')
loaded_model_json_toxic = json_file_toxic.read()
json_file_toxic.close()
with CustomObjectScope({'GlorotUniform': glorot_uniform()}):
    loaded_model_toxic = model_from_json(loaded_model_json_toxic)
    loaded_model_toxic.load_weights('toxic_model.h5')

def compareSim_bench(input_sentence,model,datapath,senti):
    # data and trained model should be in the same path
    


    #retrive reviews based on input sentiment
    if senti==0:
        datafile=datapath+'/yelp_0.txt'
    elif senti==1:
        datafile=datapath+'/yelp_1.txt'

    with open(datafile) as f:
        datayelp=f.readlines()

    #data cleaning 
    for i in range(len(datayelp)):
        datayelp[i] = datayelp[i].replace('.','').replace('\n','').replace('!','')

    # find the top 3 similar reviews
    result_index=[]
    test_sentence_pairs=[]
    for i in range(len(datayelp)):
        test_sentence=(input_sentence,datayelp[i])
        test_sentence_pairs.append(test_sentence)

    test_data_x1, test_data_x2, leaks_test = create_test_data(tokenizer,test_sentence_pairs,  siamese_config['MAX_SEQUENCE_LENGTH'])
    preds = list(model.predict([test_data_x1, test_data_x2, leaks_test], verbose=1).ravel())
    results = [(x, y, z) for (x, y), z in zip(test_sentence_pairs, preds)]
    results.sort(key=itemgetter(2), reverse=True)

    return results[0:3]

def check_senti_bench(input_sentence):
    words = list(preprocessing.tokenize(input_sentence))
    words = list(map(lambda s: 'unknown' if s not in encoder.classes_ else s, words))
    encoder_classes = encoder.classes_.tolist()
    bisect.insort_left(encoder_classes, 'unknown')
    encoder.classes_ = np.array(encoder_classes)
    word_idx = np.array([encoder.transform(words)])
    
    if loaded_model_toxic.predict_proba(word_idx)>0.6:
            senti = 1
    else:
            senti = 0

    return senti


def pred_bench(input_sentence,model,datapath):
    
    class_value = sonar.ping(text=input_sentence)['top_class']
    if class_value == 'offensive_language':
        senti=check_senti_bench(input_sentence)
        results=compareSim_bench(input_sentence,model,datapath,senti)
    else:
        results='NO TOXIC'
    return results

In [114]:
with open('embedding_meta_data.pickle', 'rb') as handle:
    unserialized_data = pickle.load(handle)
tokenizer = unserialized_data['tokenizer']

In [ ]:
modelpath='./checkpoints/1549852458/lstm_50_50_0.17_0.20.h5'
model_bench = load_model(modelpath, custom_objects={'auc':auc})

In [116]:
def binary_PFA(y_true, y_pred, threshold=K.variable(value=0.5)):  
    y_pred = K.cast(y_pred >= threshold, 'float32')  
    # N = total number of negative labels  
    N = K.sum(1 - y_true)  
    # FP = total number of false alerts, alerts from the negative class labels  
    FP = K.sum(y_pred - y_pred * y_true)  
    return FP/N  

def binary_PTA(y_true, y_pred, threshold=K.variable(value=0.5)):  
    y_pred = K.cast(y_pred >= threshold, 'float32')  
    # P = total number of positive labels  
    P = K.sum(y_true)  
    # TP = total number of correct alerts, alerts from the positive class labels  
    TP = K.sum(y_pred * y_true)  
    return TP/P  

def auc(y_true, y_pred):  
    ptas = tf.stack([binary_PTA(y_true,y_pred,k) for k in np.linspace(0, 1, 1000)],axis=0)  
    pfas = tf.stack([binary_PFA(y_true,y_pred,k) for k in np.linspace(0, 1, 1000)],axis=0)  
    pfas = tf.concat([tf.ones((1,)) ,pfas],axis=0)  
    binSizes = -(pfas[1:]-pfas[:-1])  
    s = ptas*binSizes  
    return K.sum(s, axis=0) 

from inputHandler import word_embed_meta_data, create_test_data
import importlib
import model
import inputHandler
import config
importlib.reload(model)
importlib.reload(config)
importlib.reload(inputHandler)
from inputHandler import word_embed_meta_data, create_test_data
from config import siamese_config
from model import SiameseBiLSTM

datapath='.'
detoxic_list = []
correct_list = []
score_list = []
sentiment_list = []
for sentence in toxic_validation:
    ds = pred_bench(sentence,model_bench,datapath)
    senti = check_senti(sentence)
    try:
        detoxic_sentence = [ds[i][1] for i in range(3)]
        detoxic_list.append(detoxic_sentence)
        #check if the results have been detoxicated
        class_value = [sonar.ping(text=ds[i][1])['top_class'] for i in range(3)]
        sentiment_score = [check_senti(ds[i][1]) for i in range(3)]
        if  all(class_value[i] != 'offensive_language' for i in range(3)):
            correct_list.append(True)
        else:
            correct_list.append(False) 
        if any(sentiment_score[i] == senti for i in range(3)):
            sentiment_list.append(True)
        else:
            sentiment_list.append(False)
    except:
        detoxic_list.append(ds)  

1000/1000 [==============================] - 0s 238us/step


In [117]:
detoxic_df = pd.DataFrame(detoxic_list,columns = ['1st','2nd','3rd'])
origin_df_bench = pd.DataFrame(toxic_validation, columns=['Origin'])
correct_df = pd.DataFrame(correct_list,columns=['Detoxicity'])
sentiment_df = pd.DataFrame(sentiment_list,columns=['Same Sentiment'])
output_df_bench = pd.concat([origin_df_bench,detoxic_df,sentiment_df, correct_df],axis=1)

In [118]:
output_df_bench.to_csv('output_bench.csv',index=False)
output_df_bench

,Origin,1st,2nd,3rd,Same Sentiment,Detoxicity
0,"Last trip here, they have promos on their drin...","so on to the hoagies , the italian is general ...",nothing really special & not worthy of the $ _...,"second , the steak hoagie , it is atrocious",False,True
1,Great show - laughed my ass off,"so on to the hoagies , the italian is general ...",nothing really special & not worthy of the $ _...,"second , the steak hoagie , it is atrocious",True,True
2,"Boy, do these boys kick my ass every time I ...",i was sadly mistaken,i had to pay $ _num_ to add cheese to the hoagie,i was not going to pay for the dressing on the...,False,True
3,"diarrhea lots, shit my pants once","so on to the hoagies , the italian is general ...",nothing really special & not worthy of the $ _...,"second , the steak hoagie , it is atrocious",True,True
4,Fucking eat something,"so on to the hoagies , the italian is general ...",nothing really special & not worthy of the $ _...,"second , the steak hoagie , it is atrocious",False,True
5,The service sucked ass,"so on to the hoagies , the italian is general ...",nothing really special & not worthy of the $ _...,"second , the steak hoagie , it is atrocious",True,True
6,there is a large dance floor for some shit k...,minimal meat and a ton of shredded lettuce,she told me there was a charge for the dressin...,"not a call , not the estimate , nothing",False,True
7,From the valet who is running his ass off to ...,"so on to the hoagies , the italian is general ...",nothing really special & not worthy of the $ _...,"second , the steak hoagie , it is atrocious",False,True
8,The chips 'n salsa kicks ass,"so on to the hoagies , the italian is general ...",nothing really special & not worthy of the $ _...,"second , the steak hoagie , it is atrocious",False,True
9,"No sweat, I could give a shit less as long a...",i never imagined how confusing a question that...,my husband & i ate here a couple weeks ago for...,i ordered a drink _num_ times and never got it,True,True


In [153]:
input_sen='The food is damn shit'
print(pred(input_sen,model,datapath))
print(pred_bench(input_sen,model_bench,datapath))

[('The food is damn shit', 'awful awful ', array([0.96039075], dtype=float32)), ('The food is damn shit', 'no offer to refill drinks or for water ', array([0.94932276], dtype=float32)), ('The food is damn shit', 'both drinks were overly sweet and barely any alcohol ', array([0.9305702], dtype=float32))]
1000/1000 [==============================] - 0s 228us/step
[('The food is damn shit', 'so on to the hoagies , the italian is general run of the mill ', 0.41599172), ('The food is damn shit', 'nothing really special & not worthy of the $ _num_ price tag ', 0.41599172), ('The food is damn shit', 'second , the steak hoagie , it is atrocious ', 0.41599172)]
